In [3]:
# Unified Router caller (same for all tests)
import requests, json

AGENT_URL = "http://127.0.0.1:8000/collect"  # ← change if different

def call_router(intent: str, args: dict | None = None):
    r = requests.post(AGENT_URL, json={"intent": intent, "args": args or {}}, timeout=60)
    r.raise_for_status()
    resp = r.json()
    print("OK:", resp.get("ok"), "| Intent:", resp.get("intent"))
    if not resp.get("ok"):
        print(json.dumps(resp, indent=2))
        raise SystemExit
    # Show which provider actually answered
    src = (resp.get("meta") or {}).get("source") or {}
    print("Provider(s):", src)
    # Return whole response so you can inspect raw as needed
    return resp

def show_first(obj, n=3):
    if isinstance(obj, list):
        for i, x in enumerate(obj[:n], 1):
            print(f"{i:>2}.", (x if isinstance(x, (str,int,float)) else {k:x.get(k) for k in list(x)[:8]}))
    elif isinstance(obj, dict):
        print({k: obj.get(k) for k in list(obj)[:12]})
    else:
        print(repr(obj))

In [ ]:

resp = call_router("leagues.list")
data = resp["data"]
print("Keys:", list(data.keys()))
leagues = data.get("leagues") or data.get("result") or []
print("Total leagues:", len(leagues))
show_first(leagues, n=15)

OK: True | Intent: leagues.list
Provider(s): {'primary': 'allsports', 'fallback': None}
Keys: ['success', 'result']
Total leagues: 976
 1. {'league_key': 4, 'league_name': 'UEFA Europa League', 'country_key': 1, 'country_name': 'eurocups', 'league_logo': 'https://apiv2.allsportsapi.com/logo/logo_leagues/', 'country_logo': None}
 2. {'league_key': 1, 'league_name': 'UEFA European Championship', 'country_key': 1, 'country_name': 'eurocups', 'league_logo': None, 'country_logo': None}
 3. {'league_key': 683, 'league_name': 'UEFA Conference League', 'country_key': 1, 'country_name': 'eurocups', 'league_logo': None, 'country_logo': None}
 4. {'league_key': 3, 'league_name': 'UEFA Champions League', 'country_key': 1, 'country_name': 'eurocups', 'league_logo': 'https://apiv2.allsportsapi.com/logo/logo_leagues/3_uefa_champions_league.png', 'country_logo': None}
 5. {'league_key': 633, 'league_name': 'UEFA Nations League', 'country_key': 1, 'country_name': 'eurocups', 'league_logo': None, 'count

In [1]:
import os, requests, json

API_KEY = os.getenv("ALLSPORTS_API_KEY")
if not API_KEY:
    raise RuntimeError("ALLSPORTS_API_KEY not found in environment")

BASE_URL = "https://apiv2.allsportsapi.com/football/"

def call_api(met, **params):
    """Helper to call a football endpoint"""
    q = {"met": met, "APIkey": API_KEY}
    q.update(params)
    r = requests.get(BASE_URL, params=q, timeout=10)
    data = r.json()
    print(json.dumps(data, indent=2))
    return data

In [8]:
# Returns league table standings
call_api("Standings", leagueId=152)

{
  "success": 1,
  "result": {
    "total": [
      {
        "standing_place": 1,
        "standing_place_type": "Promotion - Champions League (League phase: )",
        "standing_team": "Arsenal",
        "standing_P": 8,
        "standing_W": 6,
        "standing_D": 1,
        "standing_L": 1,
        "standing_F": 15,
        "standing_A": 3,
        "standing_GD": 12,
        "standing_PTS": 19,
        "team_key": 141,
        "league_key": 152,
        "league_season": "2025/2026",
        "league_round": "",
        "standing_updated": "2025-10-22 09:27:12",
        "fk_stage_key": 6,
        "stage_name": "Current",
        "team_logo": "https://apiv2.allsportsapi.com/logo/141_arsenal-fc.jpg",
        "standing_LP": 0,
        "standing_WP": 0
      },
      {
        "standing_place": 2,
        "standing_place_type": "Promotion - Champions League (League phase: )",
        "standing_team": "Manchester City",
        "standing_P": 8,
        "standing_W": 5,
        "standi

{'success': 1,
 'result': {'total': [{'standing_place': 1,
    'standing_place_type': 'Promotion - Champions League (League phase: )',
    'standing_team': 'Arsenal',
    'standing_P': 8,
    'standing_W': 6,
    'standing_D': 1,
    'standing_L': 1,
    'standing_F': 15,
    'standing_A': 3,
    'standing_GD': 12,
    'standing_PTS': 19,
    'team_key': 141,
    'league_key': 152,
    'league_season': '2025/2026',
    'league_round': '',
    'standing_updated': '2025-10-22 09:27:12',
    'fk_stage_key': 6,
    'stage_name': 'Current',
    'team_logo': 'https://apiv2.allsportsapi.com/logo/141_arsenal-fc.jpg',
    'standing_LP': 0,
    'standing_WP': 0},
   {'standing_place': 2,
    'standing_place_type': 'Promotion - Champions League (League phase: )',
    'standing_team': 'Manchester City',
    'standing_P': 8,
    'standing_W': 5,
    'standing_D': 1,
    'standing_L': 2,
    'standing_F': 17,
    'standing_A': 6,
    'standing_GD': 11,
    'standing_PTS': 16,
    'team_key': 80,
   

In [ ]:
import csv

# Assuming you already ran:
# resp = call_router("leagues.list")
# data = resp["data"]
# leagues = data.get("leagues") or data.get("result") or []

output_file = "leagues_list.csv"

with open(output_file, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    # header row
    writer.writerow(["league_key", "league_name", "country_name"])
    # data rows
    for l in leagues:
        writer.writerow([
            l.get("league_key"),
            l.get("league_name"),
            l.get("country_name"),
        ])

print(f"✅ Saved {len(leagues)} leagues to {output_file}")

✅ Saved 976 leagues to leagues_list.csv
